<a href="https://colab.research.google.com/github/HanyNaseer/ETLPipelineWithDocker/blob/main/Online_Retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load the Dataset**

In [ ]:
import pandas as pd

# Load the dataset with ISO-8859-1 encoding
df = pd.read_csv('/content/OnlineRetail.csv', encoding='ISO-8859-1')

# Take a quick look at the dataset
print(df.head())


**Data Cleaning**

In [ ]:
# Check for missing values
print(df.isnull().sum())

In [27]:
# Drop missing CustomerID and Description rows

df_cleaned = df.dropna(subset=['CustomerID', 'Description'])

In [28]:
# Remove duplicates

df_cleaned = df_cleaned.drop_duplicates()

**Perform Basic Analysis**

In [ ]:
# Total Sales per Country:
# Add a new column that calculates total sales (UnitPrice * Quantity), then group the data by country.

# Add a TotalSales column
df_cleaned['TotalSales'] = df_cleaned['UnitPrice'] * df_cleaned['Quantity']

# Group by Country and calculate total sales
sales_by_country = df_cleaned.groupby('Country')['TotalSales'].sum().sort_values(ascending=False)
print(sales_by_country)

In [ ]:
# Top 10 Best-Selling Products
# Find the best-selling products based on quantity sold.

best_selling = df_cleaned.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
print(best_selling)

In [ ]:
# Revenue Over Time
# Analyze the revenue by grouping the data by InvoiceDate and looking at trends over time.

# Convert InvoiceDate to datetime
df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'])

# Group by month and calculate total sales
df_cleaned['Month'] = df_cleaned['InvoiceDate'].dt.to_period('M')
revenue_by_month = df_cleaned.groupby('Month')['TotalSales'].sum()

print(revenue_by_month)

**Customer Analysis**

In [ ]:
# Who are the top buyers?
# To find the top buyers, we’ll group the data by CustomerID and sum up their total spending.

# Add a TotalSales column if it's not already added
df['TotalSales'] = df['UnitPrice'] * df['Quantity']

# Group by CustomerID and calculate total spending
top_buyers = df.groupby('CustomerID')['TotalSales'].sum().sort_values(ascending=False).head(10)
print("Top 10 Buyers:")
print(top_buyers)

In [ ]:
# How often do they return?
# To calculate how often a customer returns, we can count the number of unique invoices for each customer.

# Group by CustomerID and count unique InvoiceNo to see how many times each customer made a purchase
customer_frequency = df.groupby('CustomerID')['InvoiceNo'].nunique().sort_values(ascending=False).head(10)
print("Top 10 Most Frequent Customers:")
print(customer_frequency)

In [ ]:
# What is the average spend?
# We can calculate the average spend per transaction for each customer.

# Calculate average spend per transaction for each customer
avg_spend = df.groupby('CustomerID')['TotalSales'].mean().sort_values(ascending=False).head(10)
print("Top 10 Customers by Average Spend:")
print(avg_spend)

**Returns Analysis**

In [35]:

# How much of the products are returned?
# We'll calculate the total value of returns and compare it to total sales.

# Filter out rows where Quantity is negative (returns)
returns = df[df['Quantity'] < 0]

# Calculate total returns and total sales
total_returns = returns['TotalSales'].sum()
total_sales = df['TotalSales'].sum()

# Percentage of returns
return_percentage = abs(total_returns) / total_sales * 100
print(f"Total Returns: {total_returns}")
print(f"Percentage of Returns: {return_percentage:.2f}%")

Total Returns: -896812.49
Percentage of Returns: 9.20%


**Time-Series Trends**

In [ ]:
# Daily, Weekly, and Monthly Sales Trends
# We can analyze sales trends by grouping the data by day, week, or month. Let’s start by converting the InvoiceDate column to a datetime format (if not already done), and then group by different time periods.

# Ensure InvoiceDate is in datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Group by day and calculate total sales
daily_sales = df.groupby(df['InvoiceDate'].dt.date)['TotalSales'].sum()
print("Daily Sales:")
print(daily_sales)

# Group by week and calculate total sales
weekly_sales = df.groupby(df['InvoiceDate'].dt.to_period('W'))['TotalSales'].sum()
print("Weekly Sales:")
print(weekly_sales)

# Group by month and calculate total sales
monthly_sales = df.groupby(df['InvoiceDate'].dt.to_period('M'))['TotalSales'].sum()
print("Monthly Sales:")
print(monthly_sales)

**Visualize the Data**

In [ ]:
# Visualize Total Sales by Country:
import matplotlib.pyplot as plt

# Plot total sales by country
sales_by_country.plot(kind='bar', figsize=(10, 6))
plt.title('Total Sales by Country')
plt.ylabel('Total Sales')
plt.xlabel('Country')
plt.show()


In [ ]:
# Top 10 Customers by Purchase Frequency
customer_frequency = df.groupby('CustomerID')['InvoiceNo'].nunique().sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 6))
customer_frequency.plot(kind='bar', color='salmon')
plt.title("Top 10 Most Frequent Customers")
plt.xlabel("Customer ID")
plt.ylabel("Number of Purchases")
plt.show()


In [ ]:
# Top 10 Customers by Average Spend per Transaction
avg_spend = df.groupby('CustomerID')['TotalSales'].mean().sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 6))
avg_spend.plot(kind='bar', color='lightgreen')
plt.title("Top 10 Customers by Average Spend per Transaction")
plt.xlabel("Customer ID")
plt.ylabel("Average Spend per Transaction")
plt.show()

In [ ]:
# Top 10 Buyers by Total Spending
top_buyers = df.groupby('CustomerID')['TotalSales'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 6))
top_buyers.plot(kind='bar', color='skyblue')
plt.title("Top 10 Buyers by Total Spending")
plt.xlabel("Customer ID")
plt.ylabel("Total Spending")
plt.show()

In [ ]:
# Group by month to get monthly revenue
monthly_revenue = df.groupby(df['InvoiceDate'].dt.to_period('M'))['TotalSales'].sum()

plt.figure(figsize=(12, 6))
monthly_revenue.plot(kind='line', color='blue')
plt.title("Revenue Over Time (Monthly)")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.grid(True)
plt.show()

In [ ]:
# Returns Analysis - Pie chart of Returns vs Sales
returns = df[df['Quantity'] < 0]['TotalSales'].sum()
total_sales = df['TotalSales'].sum()
sales_vs_returns = [abs(returns), total_sales - abs(returns)]

plt.figure(figsize=(8, 8))
plt.pie(sales_vs_returns, labels=['Returns', 'Sales'], autopct='%1.1f%%', colors=['#ff9999','#66b3ff'])
plt.title("Percentage of Returns vs Sales")
plt.show()

In [ ]:
# Plot daily sales
plt.figure(figsize=(10, 6))
daily_sales.plot()
plt.title('Daily Sales Over Time')
plt.ylabel('Total Sales')
plt.xlabel('Date')
plt.show()

# Plot monthly sales
plt.figure(figsize=(10, 6))
monthly_sales.plot()
plt.title('Monthly Sales Over Time')
plt.ylabel('Total Sales')
plt.xlabel('Month')
plt.show()